# Using Bayesian Optimization for tuning the hyperparameters
- data set of application and prev.application

# Loading libraries and datasets. 
- Copying FE functions from previous application data

In [3]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [4]:
def get_dataset():
    app_train = pd.read_csv('dataset/application_train.csv')
    app_test = pd.read_csv('dataset/application_test.csv')
    apps = pd.concat([app_train, app_test])
    prev = pd.read_csv('dataset/previous_application.csv')

    return apps, prev

apps, prev = get_dataset()
    

# Copy feature engineering function from previous application data

In [5]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X 
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
      
    # AMT_CREDIT 
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
       
    return apps

# Declaration of functions that encode and generate the final set of data after processing the previous application data

In [6]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

def get_prev_processed(prev):
    # The difference and ratio between the loan application amount and the actual loan amount/loan product amount
    prev['PREV_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['PREV_GOODS_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_GOODS_PRICE']
    prev['PREV_CREDIT_APPL_RATIO'] = prev['AMT_CREDIT']/prev['AMT_APPLICATION']
    # prev['PREV_ANNUITY_APPL_RATIO'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION']
    prev['PREV_GOODS_APPL_RATIO'] = prev['AMT_GOODS_PRICE']/prev['AMT_APPLICATION']
    
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # the period from the first due date to the last due date
    prev['PREV_DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
    # Calculate the total amount paid by multiplying the monthly amount by the number of payments.
    all_pay = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    # Calculate the interest rate by obtaining the AMT_CREDIT ratio to the total amount paid and dividing it by the number of payments again.
    prev['PREV_INTERESTS_RATE'] = (all_pay/prev['AMT_CREDIT'] - 1)/prev['CNT_PAYMENT']
        
    return prev
    
    
def get_prev_amt_agg(prev):
    # Aggregation is carried out with a difference and ratio of different amounts compared to the newly created loan application amount.
    agg_dict = {
         # aggregation with the existing columns
        'SK_ID_CURR':['count'],
        'AMT_CREDIT':['mean', 'max', 'sum'],
        'AMT_ANNUITY':['mean', 'max', 'sum'], 
        'AMT_APPLICATION':['mean', 'max', 'sum'],
        'AMT_DOWN_PAYMENT':['mean', 'max', 'sum'],
        'AMT_GOODS_PRICE':['mean', 'max', 'sum'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        # aggregation with the pre-processed columns
        'PREV_CREDIT_DIFF':['mean', 'max', 'sum'], 
        'PREV_CREDIT_APPL_RATIO':['mean', 'max'],
        'PREV_GOODS_DIFF':['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO':['mean', 'max'],
        'PREV_DAYS_LAST_DUE_DIFF':['mean', 'max', 'sum'],
        'PREV_INTERESTS_RATE':['mean', 'max']
    }

    prev_group = prev.groupby('SK_ID_CURR')
    prev_amt_agg = prev_group.agg(agg_dict)

    # change col names with underbar _
    prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]
    
    return prev_amt_agg

def get_prev_refused_appr_agg(prev):
    # Perform groupby with the original groupby column + detailed reference column. Aggregates at a granular level and then transforms to an unstack() column level.
    prev_refused_appr_group = prev[prev['NAME_CONTRACT_STATUS'].isin(['Approved', 'Refused'])].groupby([ 'SK_ID_CURR', 'NAME_CONTRACT_STATUS'])
    prev_refused_appr_agg = prev_refused_appr_group['SK_ID_CURR'].count().unstack()
    # change col names
    prev_refused_appr_agg.columns = ['PREV_APPROVED_COUNT', 'PREV_REFUSED_COUNT' ]
    # NaN into 0
    prev_refused_appr_agg = prev_refused_appr_agg.fillna(0)
    
    return prev_refused_appr_agg

    

def get_prev_agg(prev):
    prev = get_prev_processed(prev)
    prev_amt_agg = get_prev_amt_agg(prev)
    prev_refused_appr_agg = get_prev_refused_appr_agg(prev)
    prev_days365_agg = get_prev_days365_agg(prev)
    
    # join with prev_amt_agg
    prev_agg = prev_amt_agg.merge(prev_refused_appr_agg, on='SK_ID_CURR', how='left')
    prev_agg = prev_agg.merge(prev_days365_agg, on='SK_ID_CURR', how='left')
    # Generation of APPROVED_COUNT and REFUSED_COUNT ratio compared to past loans by SK_ID_CURR.
    prev_agg['PREV_REFUSED_RATIO'] = prev_agg['PREV_REFUSED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
    prev_agg['PREV_APPROVED_RATIO'] = prev_agg['PREV_APPROVED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
    # Drop 'PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT'
    prev_agg = prev_agg.drop(['PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT'], axis=1)
    
    return prev_agg

def get_apps_all_with_prev_agg(apps, prev):
    apps_all =  get_apps_processed(apps)
    prev_agg = get_prev_agg(prev)
    print('prev_agg shape:', prev_agg.shape)
    print('apps_all before merge shape:', apps_all.shape)
    apps_all = apps_all.merge(prev_agg, on='SK_ID_CURR', how='left')
    print('apps_all after merge with prev_agg shape:', apps_all.shape)
    
    return apps_all

def get_apps_all_encoded(apps_all):
    object_columns = apps_all.dtypes[apps_all.dtypes == 'object'].index.tolist()
    for column in object_columns:
        apps_all[column] = pd.factorize(apps_all[column])[0]
    
    return apps_all

def get_apps_all_train_test(apps_all):
    apps_all_train = apps_all[~apps_all['TARGET'].isnull()]
    apps_all_test = apps_all[apps_all['TARGET'].isnull()]

    apps_all_test = apps_all_test.drop('TARGET', axis=1)
    
    return apps_all_train, apps_all_test
    
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=2000,
                learning_rate=0.01,
                num_leaves=32,
                colsample_bytree=0.8,
                subsample=0.8,
                max_depth=8,
                reg_alpha=0.04,
                reg_lambda=0.07,
                min_child_weight=40,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

# Encoding, Splitting train and test, and Splitting train and valid before the final dataset

In [7]:
apps_all = get_apps_all_with_prev_agg(apps, prev)
apps_all = get_apps_all_encoded(apps_all)
apps_all_train, apps_all_test = get_apps_all_train_test(apps_all)
ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_all_train['TARGET']
train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)


/var/folders/df/19n86p1s63b0654c8m28bsg80000gn/T/ipykernel_68453/1480129953.py:53: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]


prev_agg shape: (338857, 41)
apps_all before merge shape: (356255, 135)
apps_all after merge with prev_agg shape: (356255, 176)


#### Bayesian Optimization 

In [8]:
# bayesian optimization 패키지 설치
!conda install bayesian-optimization

In [9]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

# Set the range of testing per each hyper-parameter

In [10]:
# parameter 별로 search할 범위를 설정. 
bayesian_params = {
    'max_depth': (6, 16), 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

# Declaration of the function to get the best roc_auc_score
- Enter hyperparameter at every iteration to learn the classifier and return the loc_auc_score value

In [11]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":500, "learning_rate":0.02,
        'max_depth': int(round(max_depth)), #  Change integer hyperparameters to integer types because real-world values are entered at the time of call
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)
    
    return roc_auc   

# Bayesian Optimization object
- The object performs iterations for search of input value with maximum function return value

In [12]:
#  Create Bayesian Optimization by setting the function to perform the object and the parameter range to search.
lgbBO = BayesianOptimization(lgb_roc_eval,bayesian_params , random_state=0)
# Perform etherification to infer the input value whose function return value is maximum.
lgbBO.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.769939	training's binary_logloss: 0.245972	valid_1's auc: 0.755365	valid_1's binary_logloss: 0.248955
[200]	training's auc: 0.787803	training's binary_logloss: 0.238321	valid_1's auc: 0.766241	valid_1's binary_logloss: 0.244245
[300]	training's auc: 0.799182	training's binary_logloss: 0.233888	valid_1's auc: 0.771322	valid_1's binary_logloss: 0.2424
[400]	training's auc: 0.808279	training's binary_logloss: 0.230482	valid_1's auc: 0.773931	valid_1's binary_logloss: 0.241467
[500]	training's auc: 0.816375	training's binary_logloss: 0.227514	valid_1's auc: 0.775402	valid_1's binary_logloss: 0.240963
|  1        |  0.7754   |  0.7744   |  360.4    |  12.03    |  113.5    |  21.76    |  49.84    |  21.88    |  8.918    |  0.9818   |


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.762698	training's binary_logloss: 0.247398	valid_1's auc: 0.753688	valid_1's binary_logloss: 0.249103
[200]	training's auc: 0.780369	training's binary_logloss: 0.240433	valid_1's auc: 0.765612	valid_1's binary_logloss: 0.2443
[300]	training's auc: 0.791026	training's binary_logloss: 0.236421	valid_1's auc: 0.771186	valid_1's binary_logloss: 0.242295
[400]	training's auc: 0.7992	training's binary_logloss: 0.2335	valid_1's auc: 0.773979	valid_1's binary_logloss: 0.241311
[500]	training's auc: 0.806039	training's binary_logloss: 0.231038	valid_1's auc: 0.775665	valid_1's binary_logloss: 0.240713
|  2        |  0.7757   |  0.6917   |  397.9    |  11.29    |  117.9    |  46.35    |  26.84    |  4.366    |  0.2032   |  0.9163   |


...

[100]	training's auc: 0.771861	training's binary_logloss: 0.244863	valid_1's auc: 0.757265	valid_1's binary_logloss: 0.248069
[200]	training's auc: 0.791359	training's binary_logloss: 0.236792	valid_1's auc: 0.768042	valid_1's binary_logloss: 0.243445
[300]	training's auc: 0.80466	training's binary_logloss: 0.231733	valid_1's auc: 0.772717	valid_1's binary_logloss: 0.241718
[400]	training's auc: 0.815551	training's binary_logloss: 0.227692	valid_1's auc: 0.77547	valid_1's binary_logloss: 0.240779
[500]	training's auc: 0.824795	training's binary_logloss: 0.224284	valid_1's auc: 0.776971	valid_1's binary_logloss: 0.240268
|  28       |  0.777    |  0.7844   |  497.8    |  11.24    |  48.74    |  41.13    |  41.77    |  1.772    |  4.538    |  0.9283   |


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.778787	training's binary_logloss: 0.243316	valid_1's auc: 0.759673	valid_1's binary_logloss: 0.247687
[200]	training's auc: 0.798839	training's binary_logloss: 0.234367	valid_1's auc: 0.769386	valid_1's binary_logloss: 0.24303
[300]	training's auc: 0.814555	training's binary_logloss: 0.228362	valid_1's auc: 0.773898	valid_1's binary_logloss: 0.241323
[400]	training's auc: 0.827583	training's binary_logloss: 0.223427	valid_1's auc: 0.776396	valid_1's binary_logloss: 0.240454
[500]	training's auc: 0.838887	training's binary_logloss: 0.21912	valid_1's auc: 0.777674	valid_1's binary_logloss: 0.239993
|  29       |  0.7777   |  0.6173   |  499.4    |  13.45    |  60.94    |  23.23    |  56.33    |  4.316    |  2.0      |  0.6298   |


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.774989	training's binary_logloss: 0.244873	valid_1's auc: 0.7579	valid_1's binary_logloss: 0.248484
[200]	training's auc: 0.792792	training's binary_logloss: 0.236701	valid_1's auc: 0.767601	valid_1's binary_logloss: 0.243744
[300]	training's auc: 0.806074	training's binary_logloss: 0.231522	valid_1's auc: 0.772761	valid_1's binary_logloss: 0.241832
[400]	training's auc: 0.816895	training's binary_logloss: 0.227504	valid_1's auc: 0.775096	valid_1's binary_logloss: 0.240998
[500]	training's auc: 0.826579	training's binary_logloss: 0.22387	valid_1's auc: 0.776779	valid_1's binary_logloss: 0.240405
|  30       |  0.7768   |  0.6016   |  498.3    |  14.95    |  45.87    |  23.19    |  55.85    |  12.81    |  8.804    |  0.9214   |


# Output of the Iteration action result

In [13]:
# The res of the Bayesian Optimization object has all function return results and parameter result values at that time whenever it is performed.
lgbBO.res

[{'target': 0.7754019849993636,
  'params': {'colsample_bytree': 0.7744067519636624,
   'max_bin': 360.44278952248555,
   'max_depth': 12.027633760716439,
   'min_child_samples': 113.52780476941041,
   'min_child_weight': 21.75908516760633,
   'num_leaves': 49.835764522666246,
   'reg_alpha': 21.884984691022,
   'reg_lambda': 8.917838234820016,
   'subsample': 0.9818313802505146}},
 {'target': 0.7756649587625722,
  'params': {'colsample_bytree': 0.6917207594128889,
   'max_bin': 397.94526866050563,
   'max_depth': 11.288949197529044,
   'min_child_samples': 117.92846660784714,
   'min_child_weight': 46.35423527634039,
   'num_leaves': 26.841442327915477,
   'reg_alpha': 4.36559369208002,
   'reg_lambda': 0.20316375600581688,
   'subsample': 0.916309922773969}},
 {'target': 0.7773337907658435,
  'params': {'colsample_bytree': 0.8890783754749252,
   'max_bin': 436.30595264094137,
   'max_depth': 15.78618342232764,
   'min_child_samples': 161.8401272011775,
   'min_child_weight': 23.61248

# As a result of the Iteration
- the index with the maximum target value is extracted from the Dictionary and the parameter value is extracted.

In [14]:
# Extract all target values in dictionary
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# Extract the index with the largest target value
print('maximum target index:', np.argmax(np.array(target_list)))

[0.7754019849993636, 0.7756649587625722, 0.7773337907658435, 0.7743404274231133, 0.7740445759828272, 0.7741741249590283, 0.7773465253458576, 0.7755691213981508, 0.7773196456448476, 0.7762113766061457, 0.7762440223699537, 0.7772812798832063, 0.7761326579356012, 0.7773153203035257, 0.7770684120937986, 0.7749555583092521, 0.7764718835110829, 0.7776325792447284, 0.7769369951038645, 0.7773433992820599, 0.7768711555620157, 0.7766344976011137, 0.7771676535002521, 0.7761250905738466, 0.7772459512323278, 0.7768328263346566, 0.776410856963801, 0.7769705589900684, 0.7776743569906369, 0.7767790351636954]
maximum target index: 28


In [15]:
# Extract the corresponding parameter from res based on index value with the largest target value.
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.7776743569906369, 'params': {'colsample_bytree': 0.6173018328580049, 'max_bin': 499.3972330245431, 'max_depth': 13.449824828941859, 'min_child_samples': 60.93736887309616, 'min_child_weight': 23.22575665016526, 'num_leaves': 56.32535872887304, 'reg_alpha': 4.316459771428574, 'reg_lambda': 2.0002145324686245, 'subsample': 0.6297763732797832}}


# Retest based on optimized hyperparameters

In [16]:
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=1000,
                learning_rate=0.02,
                max_depth = 13,
                num_leaves=57,
                colsample_bytree=0.638,
                subsample=0.682,
                max_bin=435,
                reg_alpha=0.936,
                reg_lambda=4.533,
                min_child_weight=25,
                min_child_samples=166,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

In [17]:
apps_all = get_apps_all_with_prev_agg(apps, prev)
apps_all = get_apps_all_encoded(apps_all)
apps_all_train, apps_all_test = get_apps_all_train_test(apps_all)
clf = train_apps_all(apps_all_train)

/var/folders/df/19n86p1s63b0654c8m28bsg80000gn/T/ipykernel_68453/1480129953.py:53: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]


prev_agg shape: (338857, 41)
apps_all before merge shape: (356255, 135)
apps_all after merge with prev_agg shape: (356255, 176)
train shape: (215257, 174) valid shape: (92254, 174)


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is d

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
[100]	training's auc: 0.780153	training's binary_logloss: 0.242981	valid_1's auc: 0.759911	valid_1's binary_logloss: 0.24756
[200]	training's auc: 0.801589	training's binary_logloss: 0.233556	valid_1's auc: 0.769847	valid_1's binary_logloss: 0.242803
[300]	training's auc: 0.817628	training's binary_logloss: 0.227376	valid_1's auc: 0.773659	valid_1's binary_logloss: 0.241322
[400]	training's auc: 0.831406	training's binary_logloss: 0.222166	valid_1's auc: 0.776379	valid_1's binary_logloss: 0.240369
[500]	training's auc: 0.84293	training's binary_logloss: 0.217731	valid_1's auc: 0.777363	valid_1's binary_logloss: 0.240028
[600]	training's auc: 0.853247	training's binary_logloss: 0.21375	valid_1's auc: 0.777921	valid_1's binary_logloss: 0.239843
[700]	training's auc: 0.862535	training's binary_logloss: 0.210085	valid_1's auc: 0.77821	valid_1's binary_logloss: 0.239772
[80

In [18]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_baseline_tuning_01.csv', index=False)

# Re-tuning hyper parameters by CV

In [19]:
bayesian_params = {
    'max_depth': (6, 16), 
    'num_leaves': (24, 64), 
    'min_data_in_leaf': (10, 200), # min_child_samples
    'min_child_weight':(1, 50),
    'bagging_fraction':(0.5, 1.0), # subsample
    'feature_fraction': (0.5, 1.0), # colsample_bytree
    'max_bin':(10, 500),
    'lambda_l2':(0.001, 10), # reg_lambda
    'lambda_l1': (0.01, 50) # reg_alpha
}

In [20]:
import lightgbm as lgb

train_data = lgb.Dataset(data=ftr_app, label=target_app, free_raw_data=False)
def lgb_roc_eval_cv(max_depth, num_leaves, min_data_in_leaf, min_child_weight, bagging_fraction, 
                 feature_fraction,  max_bin, lambda_l2, lambda_l1):   
    params = {
        "num_iterations":500, "learning_rate":0.02,
        'early_stopping_rounds':100, 'metric':'auc',
        'max_depth': int(round(max_depth)), #  Change double type hyperparameters to integer types because double type values are entered at call time
        'num_leaves': int(round(num_leaves)), 
        'min_data_in_leaf': int(round(min_data_in_leaf)),
        'min_child_weight': int(round(min_child_weight)),
        'bagging_fraction': max(min(bagging_fraction, 1), 0), 
        'feature_fraction': max(min(feature_fraction, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'lambda_l2': max(lambda_l2,0),
        'lambda_l1': max(lambda_l1, 0)
    }
    # lightgbm cv 
    cv_result = lgb.cv(params, train_data, nfold=3, seed=0,  verbose_eval =100,  early_stopping_rounds=50, metrics=['auc'])
    return max(cv_result['auc-mean'])   

In [21]:
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.7776743569906369, 'params': {'colsample_bytree': 0.6173018328580049, 'max_bin': 499.3972330245431, 'max_depth': 13.449824828941859, 'min_child_samples': 60.93736887309616, 'min_child_weight': 23.22575665016526, 'num_leaves': 56.32535872887304, 'reg_alpha': 4.316459771428574, 'reg_lambda': 2.0002145324686245, 'subsample': 0.6297763732797832}}


In [22]:
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=1000,
                learning_rate=0.02,
                max_depth = 10,
                num_leaves=60,
                colsample_bytree=0.511,
                subsample=0.785,
                max_bin=208,
                reg_alpha=7.009,
                reg_lambda=6.579,
                min_child_weight=40,
                min_child_samples=91,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

In [23]:
apps_all = get_apps_all_with_prev_agg(apps, prev)
apps_all = get_apps_all_encoded(apps_all)
apps_all_train, apps_all_test = get_apps_all_train_test(apps_all)
clf = train_apps_all(apps_all_train)

/var/folders/df/19n86p1s63b0654c8m28bsg80000gn/T/ipykernel_68453/1480129953.py:53: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]


prev_agg shape: (338857, 41)
apps_all before merge shape: (356255, 135)
apps_all after merge with prev_agg shape: (356255, 176)
train shape: (215257, 174) valid shape: (92254, 174)


/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/chloe/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is d

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
[100]	training's auc: 0.778267	training's binary_logloss: 0.244198	valid_1's auc: 0.759489	valid_1's binary_logloss: 0.248092
[200]	training's auc: 0.796481	training's binary_logloss: 0.235631	valid_1's auc: 0.768743	valid_1's binary_logloss: 0.243337
[300]	training's auc: 0.809962	training's binary_logloss: 0.230278	valid_1's auc: 0.773131	valid_1's binary_logloss: 0.241656
[400]	training's auc: 0.821143	training's binary_logloss: 0.225997	valid_1's auc: 0.775193	valid_1's binary_logloss: 0.240905
[500]	training's auc: 0.831372	training's binary_logloss: 0.222146	valid_1's auc: 0.776484	valid_1's binary_logloss: 0.24044
[600]	training's auc: 0.840357	training's binary_logloss: 0.21869	valid_1's auc: 0.777441	valid_1's binary_logloss: 0.240083
[700]	training's auc: 0.848739	training's binary_logloss: 0.215451	valid_1's auc: 0.777972	valid_1's binary_logloss: 0.239874
[

In [24]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_baseline_tuning_02.csv', index=False)